In [1]:
import pandas as pd
import os
import sklearn as sk
from sklearn.linear_model import MultiTaskLasso, MultiTaskElasticNet, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import sklearn.metrics as skmet
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

In [2]:
os.listdir("inside_airbnb")

['.DS_Store',
 'calendar.csv',
 'listings.csv',
 'listings_summ.csv',
 'neighbourhoods.csv',
 'neighbourhoods.geo.json',
 'neighbourhoods.geojson',
 'neighbourhoods.json',
 'reviews.csv',
 'reviews_summ.csv',
 'stations.geojson']

In [3]:
listings = pd.read_csv('inside_airbnb/listings.csv')
list_summ = pd.read_csv('inside_airbnb/listings_summ.csv')
neighborhoods = pd.read_csv('inside_airbnb/neighbourhoods.csv')
reviews = pd.read_csv('inside_airbnb/reviews.csv')
reviews_summ = pd.read_csv('inside_airbnb/reviews_summ.csv')
calendar = pd.read_csv('inside_airbnb/calendar.csv')

In [17]:
calendar['date'] = pd.to_datetime(calendar['date'])

In [18]:
calendar['month'] = calendar['date'].apply(lambda x: x.month)
calendar['day'] = calendar['date'].apply(lambda x: x.day)
calendar['holiday'] = False

In [19]:
calendar

,listing_id,date,available,price,month,day,holiday
0,11204286,2017-07-05,t,$49.00,7,5,False
1,11204286,2017-07-04,t,$49.00,7,4,False
2,11204286,2017-07-03,t,$49.00,7,3,False
3,11204286,2017-07-02,t,$49.00,7,2,False
4,11204286,2017-07-01,t,$49.00,7,1,False
5,11204286,2017-06-30,t,$49.00,6,30,False
6,11204286,2017-06-29,t,$49.00,6,29,False
7,11204286,2017-06-28,t,$49.00,6,28,False
8,11204286,2017-06-27,t,$49.00,6,27,False
9,11204286,2017-06-26,t,$49.00,6,26,False
